In [1]:
import os
import pandas as pd
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()


Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
#Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-04-05 12:50:49--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  5.82MB/s    in 0.2s    

2022-04-05 12:50:50 (5.82 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [8]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","product_title", "product_category", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+
|     review_id|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|Tonsee 6 buttons ...|     Video Games|          5|            0|          0|   N|                Y|
|R3BH071QLH8QMC|Hidden Mysteries:...|     Video Games|          1|            0|          1|   N|                Y|
|R127K9NTSXA2YH|GelTabz Performan...|     Video Games|          3|            0|          0|   N|                Y|
|R32ZWUXDJPW27Q|Zero Suit Samus a...|     Video Games|          4|            0|          0|   N|                Y|
|R3AQQ4YUKJWBA6|Psyclone Recharge...|     Video Games|          1|      

In [9]:
# Filter the data to retrieve total_vote count > 20
vine_df_total = vine_df.filter("total_votes>=20")


In [12]:
# Filter the new dataFrame to retireve the rows where the number of helpful_botes/ total votes is greater thsn or equal to 50%
all_vines = vine_df_total.filter(vine_df_total['helpful_votes'] / vine_df_total['total_votes'] >.5)
all_vines.show()


+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+
|     review_id|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|ONE PIECE Pirate ...|     Video Games|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|       Madden NFL 16|     Video Games|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|Infinity 3.0 Star...|     Video Games|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|       Madden NFL 16|     Video Games|          1|           51|         65|   N|                Y|
|R3PZOXA5X1U8KW|Infinity 3.0 Star...|     Video Games|          4|      

In [17]:
new_vines = all_vines.filter(all_vines['vine'] == 'Y')
new_vines.show(5)

+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+
|     review_id|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+
|R3KKUSGFZWSUIY|Turtle Beach - Ea...|     Video Games|          5|           56|         63|   Y|                N|
|R10FO5UKKVZBK2|PDP AG7 True Wire...|     Video Games|          3|           23|         23|   Y|                N|
| RM4KSGEOR7MU1|Turtle Beach - Ea...|     Video Games|          5|           19|         24|   Y|                N|
| RG7VRMYLEXD23|Antec X-1 Cooler ...|     Video Games|          4|           22|         26|   Y|                N|
|R11O4YSCPSNL6L|Turtle Beach - Ea...|     Video Games|          3|           20|         26|   Y|                N|
+--------------+--------------------+----------------+-----------+------

In [18]:
no_vine = all_vines.filter(all_vines['vine'] == 'N')
no_vine.show(5)

+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+
|     review_id|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|ONE PIECE Pirate ...|     Video Games|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|       Madden NFL 16|     Video Games|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|Infinity 3.0 Star...|     Video Games|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|       Madden NFL 16|     Video Games|          1|           51|         65|   N|                Y|
+--------------+--------------------+----------------+-----------+------

In [28]:
# Determine the total number of reviews
total_reviews = vine_df.count()
total_reviews


1785997

In [32]:
# Filtered the dats set for only 5 star reviews
star_reviews = vine_df.filter(vine_df['star_rating'] == 5)
star_reviews.show()

+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+
|     review_id|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+
| RTIS3L2M1F5SM|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|
| R1ZV7R40OLHKD|Tonsee 6 buttons ...|     Video Games|          5|            0|          0|   N|                Y|
|R2F0POU5K6F73F|Protection for yo...|     Video Games|          5|            0|          0|   N|                Y|
|R3VNR804HYSMR6|   Nerf 3DS XL Armor|     Video Games|          5|            0|          0|   N|                Y|
| R3GZTM72WA2QH|One Piece: Pirate...|     Video Games|          5|            0|          0|   N|                Y|
|R1VTIA3JTYBY02|Metal Arms: Glitc...|     Video Games|          5|      

In [39]:
# Get the total number of 5 star reviews
total_number_reviews = star_reviews.count()


In [56]:
# Get the verified and unverified purchases
verified_purchase = star_reviews.filter(star_reviews['vine'] == 'Y')
non_verified_purchase = star_reviews.filter(star_reviews['vine'] == 'N')


In [57]:
verified_purchase.count()


729076

In [58]:
non_verified_purchase.count()

297848

In [59]:
star_reviews.count()

1026924

In [60]:
# Get the percent of purchased reviews
verified_purchase.count() / star_reviews.count()

0.7099610097728751

In [61]:
# Get the percentage of non-purchasewd reviews
non_verified_purchase.count() / star_reviews.count()

0.29003899022712487